# Part 3
For part 3 of the project you will be practicing applying an E.T.L process on your previously saved movie data. Specifically, you will create a new MySQL database after preparing the data for a relational database. You will export your database to a .sql file in your repository using MySQL Workbench.

## Specifications - Database
- Your stakeholder wants you to take the data you have been cleaning and collecting in Parts 1 & 2 of the project and wants you to create a MySQL database for them.

- Specifically, they want the data from the following files included in your database:

- Title Basics:
    - Movie ID (tconst)
    - Primary Title
    - Start Year
    - Runtime (in Minutes)
    - Genres

- Title Ratings
    - Movie ID (tconst)
    - Average Movie Rating
    - Number of Votes

- The TMDB API Results (multiple files)
    - Movie ID
    - Revenue
    - Budget
    - Certification (MPAA Rating)

- You should normalize the tables as best you can before adding them to your new database.

    - Note: an important exception to their request is that they would like you to keep all of the data from the TMDB API in 1 table together (even though it will not be perfectly normalized).
    - You only need to keep the imdb_id, revenue, budget, and certification columns

## Required Transformation Steps for Title Basics:
- Normalize Genre:

    - Convert the single string of genres from title basics into 2 new tables.
        1. title_genres: with the columns:

            - tconst
            - genre_id
        2. genres:

            - genre_id
            - genre_name
            
- Discard unnecessary information:

    - For the title basics table, drop the following columns:
          
        - "original_title" (we will use the primary title column instead)
        - "isAdult" ("Adult" will show up in the genres so this is redundant information).
        - "titleType" (every row will be a movie).
        - "genres" and other variants of genre (genre is now represented in the 2 new tables described above.
    - Do not include the title_akas table in your SQL database.
        - You have already filtered out the desired movies using this table, and the remaining data is mostly nulls and not of interest to the stakeholder.

## MySQL Database Requirements
   
   - Use sqlalchemy with Pandas to execute your SQL queries inside your notebook.

    - Create a new database on your MySQL server and call it "movies."

    - Make sure to have the following tables in your "movies" database:
    
        - title_basics
        - title_ratings
        - title_genres
        - genres
        - tmdb_data

  - Make sure to set a Primary Key for each table that isn't a joiner table (e.g., title_genres is a joiner table).
   
  - After creating each table, show the first 5 rows of that table using a SQL query.

  - Make sure to run the "SHOW TABLES" SQL query at the end of your notebook to show that all required tables have been created.

## Import Libraries

In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy.types import *
from sqlalchemy_utils import create_database, database_exists
from sqlalchemy import create_engine

## Normalize genres

### title_basics Dataframe

In [3]:
# Load the dataframe from the csv file
title_basics_df = pd.read_csv('Data/title_basics.csv.gz')

In [4]:
# Display the first (5) rows
title_basics_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,118,"Comedy,Fantasy,Romance"
1,tt0043139,movie,Life of a Beijing Policeman,Wo zhe yi bei zi,0,2013.0,120,"Drama,History"
2,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,70,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,122,Drama
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,100,"Comedy,Horror,Sci-Fi"


In [9]:
type('genres')


str

- We are going to convert these strings into lists of strings into a new 'genres_split' column.

In [10]:
# Convert strings into lists of strings in a new 'genres_split' column
title_basics_df['genres_split'] = title_basics_df['genres'].str.split(',')

In [11]:
title_basics_df['genres_split']

0           [Comedy, Fantasy, Romance]
1                     [Drama, History]
2                              [Drama]
3                              [Drama]
4             [Comedy, Horror, Sci-Fi]
                     ...              
81798                          [Drama]
81799         [Comedy, Drama, Fantasy]
81800                          [Drama]
81801    [Action, Adventure, Thriller]
81802                 [Drama, History]
Name: genres_split, Length: 81803, dtype: object

In [13]:
# Display the first (2) rows
title_basics_df.head(2)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,genres_split
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,118,"Comedy,Fantasy,Romance","[Comedy, Fantasy, Romance]"
1,tt0043139,movie,Life of a Beijing Policeman,Wo zhe yi bei zi,0,2013.0,120,"Drama,History","[Drama, History]"


- We will then use .explode() to separate the list of genres into new rows: one row for each genre a movie belonged to.

In [14]:
# Create a new dataframe with a row
# for each genre a movie belongs to by using .explode
exploded_genres_df = title_basics_df.explode('genres_split')

In [17]:
# Display the first (2) rows
exploded_genres_df.head(10)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,genres_split
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,118,"Comedy,Fantasy,Romance",Comedy
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,118,"Comedy,Fantasy,Romance",Fantasy
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,118,"Comedy,Fantasy,Romance",Romance
1,tt0043139,movie,Life of a Beijing Policeman,Wo zhe yi bei zi,0,2013.0,120,"Drama,History",Drama
1,tt0043139,movie,Life of a Beijing Policeman,Wo zhe yi bei zi,0,2013.0,120,"Drama,History",History
2,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,70,Drama,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,122,Drama,Drama
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,100,"Comedy,Horror,Sci-Fi",Comedy
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,100,"Comedy,Horror,Sci-Fi",Horror
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,100,"Comedy,Horror,Sci-Fi",Sci-Fi


- Finally, use .unique() to get the unique genres from the genres_split column

In [18]:
# Create a list of unique genres from the genres_split column
unique_genres = sorted(exploded_genres_df['genres_split'].unique())

In [19]:
# Display the list of unique genres
unique_genres

['Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Drama',
 'Family',
 'Fantasy',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western']

In [20]:
len(unique_genres)

25

### Create a new title_genres table

-  Create a new title_genres from the exploded df.
    - We only want the tconst and genres_split columns

In [22]:
# Save just tconst and genres_split as a new df
title_genres_df = exploded_genres_df[['tconst', 'genres_split']].copy()

In [23]:
# Display the first (5) rows
title_genres_df.head()

,tconst,genres_split
0,tt0035423,Comedy
0,tt0035423,Fantasy
0,tt0035423,Romance
1,tt0043139,Drama
1,tt0043139,History


### Create a genre mapper dictionary to replace string genres with integers

In [24]:
# Make a dictionary
# with list of unique genres as the key and the new integer id as values
genre_id_map = dict(zip(unique_genres, range(len(unique_genres))))

In [25]:
# Display the genre_id_map dictionary
genre_id_map

{'Action': 0,
 'Adult': 1,
 'Adventure': 2,
 'Animation': 3,
 'Biography': 4,
 'Comedy': 5,
 'Crime': 6,
 'Drama': 7,
 'Family': 8,
 'Fantasy': 9,
 'Game-Show': 10,
 'History': 11,
 'Horror': 12,
 'Music': 13,
 'Musical': 14,
 'Mystery': 15,
 'News': 16,
 'Reality-TV': 17,
 'Romance': 18,
 'Sci-Fi': 19,
 'Sport': 20,
 'Talk-Show': 21,
 'Thriller': 22,
 'War': 23,
 'Western': 24}

### Replace the string genres in title_genres with the new integer ids.
- Create a new genre_id column using the genre_map dictionary with either .map or .replace.
- Next, drop the "genres_split" column.
- The title_genres table is now ready for the MySQL database

In [26]:
# Create a genre_id column using the genre_map dictionary with .map
title_genres_df['genre_id'] = title_genres_df['genres_split'].map(genre_id_map)
# Drop the "genres_split" column
title_genres_df = title_genres_df.drop(columns='genres_split')

In [27]:
# Display the first (5) rows
title_genres_df.head()

,tconst,genre_id
0,tt0035423,5
0,tt0035423,9
0,tt0035423,18
1,tt0043139,7
1,tt0043139,11


### Convert the genre map dictionary into a dataframe

In [28]:
# Create a dataframe from the genre_map dictionary with .map
genres_df = pd.DataFrame({'genre_name': genre_id_map.keys(),
                           'genre_id': genre_id_map.values()})

In [29]:
# Display the first (5) rows
genres_df.head()

,genre_name,genre_id
0,Action,0
1,Adult,1
2,Adventure,2
3,Animation,3
4,Biography,4


## Create SQL Movies Database

### Saving the MySQL tables with tconst as the primary key.

In [45]:
with open('C:/Users/Melissa/.secret/mysql.json') as f:
    login = json.load(f)

# Display the MySQL login keys
login.keys()

dict_keys(['username', 'password'])

In [46]:
# Obtain the MySQL login username and password
with open(r"C:\Users\Melissa\.secret\mysql.json") as f:
    login = json.load(f)
# Display the MySQL login keys
login.keys()

dict_keys(['username', 'password'])